# Assignment 8: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A8-2-data.zip](A8-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [114]:
import pandas as pd

import featuretools as ft
from woodwork.logical_types import Categorical, Boolean
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

# list all rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [115]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

# could drop columns here, if needed
# Drop the first column from each dataframe
orders = orders.drop(orders.columns[0], axis=1)
order_products = order_products.drop(order_products.columns[0], axis=1)
users = users.drop(users.columns[0], axis=1)

print(users["label"].value_counts())
print(orders.shape, order_products.shape)
users.head()

label
False    628
True     139
Name: count, dtype: int64
(5997, 3) (57780, 7)


,user_id,label
0,1,False
1,2,True
2,3,False
3,7,False
4,10,False


## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [116]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    # --- Write your code below ---
    
    # Create an EntitySet
    entitySet = ft.EntitySet(id = 'customers')

    # Create an entity from the users dataframe
    entitySet = entitySet.add_dataframe(dataframe_name = 'users',
                                        dataframe = users, 
                                        index = 'user_id',
                                        logical_types = {'label': Boolean})

    # Create an entity from the orders dataframe
    entitySet = entitySet.add_dataframe(dataframe_name = 'orders',
                                        dataframe = orders,
                                        index = 'order_id',
                                        time_index = 'order_time')

    # Create an entity from the order_products dataframe
    entitySet = entitySet.add_dataframe(dataframe_name = 'order_products',
                                        dataframe = order_products,
                                        index = 'order_product_id',
                                        time_index = 'order_time',
                                        logical_types = {'reordered': Categorical})
    
    # Create a relationship between users (parent) and orders (child) on 'user_id'
    entitySet = entitySet.add_relationship('users', 'user_id', 'orders', 'user_id')

    # Create a relationship between orders (parent) and order_products (child) on 'order_id'
    entitySet = entitySet.add_relationship('orders', 'order_id', 'order_products', 'order_id')

    # return the EntitySet object
    return entitySet

In [117]:
es = load_entityset(orders, order_products, users)
es

/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11

Entityset: customers
  DataFrames:
    users [Rows: 767, Columns: 2]
    orders [Rows: 5997, Columns: 3]
    order_products [Rows: 57780, Columns: 7]
  Relationships:
    orders.user_id -> users.user_id
    order_products.order_id -> orders.order_id

### 1.2 Deep Feature Synthesis

In [118]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset = es, target_dataframe_name='users')

/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, logical_types
  warnings.warn(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index
  warnings.warn(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index, logical_types
  warnings.warn(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/featuretools/computational_backends/feature_set_calculator.py:828: FutureWarning: The provided callable <function sum at 0x106095c60

In [119]:
# output what features you generate
feature_matrix

,label,COUNT(orders),COUNT(order_products),MAX(order_products.aisle_id),MEAN(order_products.aisle_id),MIN(order_products.aisle_id),MODE(order_products.department),MODE(order_products.product_name),MODE(order_products.reordered),NUM_UNIQUE(order_products.department),NUM_UNIQUE(order_products.product_name),NUM_UNIQUE(order_products.reordered),SKEW(order_products.aisle_id),STD(order_products.aisle_id),SUM(order_products.aisle_id),MAX(orders.COUNT(order_products)),MAX(orders.MEAN(order_products.aisle_id)),MAX(orders.MIN(order_products.aisle_id)),MAX(orders.NUM_UNIQUE(order_products.department)),MAX(orders.NUM_UNIQUE(order_products.product_name)),MAX(orders.NUM_UNIQUE(order_products.reordered)),MAX(orders.SKEW(order_products.aisle_id)),MAX(orders.STD(order_products.aisle_id)),MAX(orders.SUM(order_products.aisle_id)),MEAN(orders.COUNT(order_products)),MEAN(orders.MAX(order_products.aisle_id)),MEAN(orders.MEAN(order_products.aisle_id)),MEAN(orders.MIN(order_products.aisle_id)),MEAN(orders.NUM_UNIQUE(order_products.department)),MEAN(orders.NUM_UNIQUE(order_products.product_name)),MEAN(orders.NUM_UNIQUE(order_products.reordered)),MEAN(orders.SKEW(order_products.aisle_id)),MEAN(orders.STD(order_products.aisle_id)),MEAN(orders.SUM(order_products.aisle_id)),MIN(orders.COUNT(order_products)),MIN(orders.MAX(order_products.aisle_id)),MIN(orders.MEAN(order_products.aisle_id)),MIN(orders.NUM_UNIQUE(order_products.department)),MIN(orders.NUM_UNIQUE(order_products.product_name)),MIN(orders.NUM_UNIQUE(order_products.reordered)),MIN(orders.SKEW(order_products.aisle_id)),MIN(orders.STD(order_products.aisle_id)),MIN(orders.SUM(order_products.aisle_id)),MODE(orders.DAY(order_time)),MODE(orders.MODE(order_products.department)),MODE(orders.MODE(order_products.product_name)),MODE(orders.MODE(order_products.reordered)),MODE(orders.MONTH(order_time)),MODE(orders.WEEKDAY(order_time)),MODE(orders.YEAR(order_time)),NUM_UNIQUE(orders.DAY(order_time)),NUM_UNIQUE(orders.MODE(order_products.department)),NUM_UNIQUE(orders.MODE(order_products.product_name)),NUM_UNIQUE(orders.MODE(order_products.reordered)),NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.WEEKDAY(order_time)),NUM_UNIQUE(orders.YEAR(order_time)),SKEW(orders.COUNT(order_products)),SKEW(orders.MAX(order_products.aisle_id)),SKEW(orders.MEAN(order_products.aisle_id)),SKEW(orders.MIN(order_products.aisle_id)),SKEW(orders.NUM_UNIQUE(order_products.department)),SKEW(orders.NUM_UNIQUE(order_products.product_name)),SKEW(orders.NUM_UNIQUE(order_products.reordered)),SKEW(orders.STD(order_products.aisle_id)),SKEW(orders.SUM(order_products.aisle_id)),STD(orders.COUNT(order_products)),STD(orders.MAX(order_products.aisle_id)),STD(orders.MEAN(order_products.aisle_id)),STD(orders.MIN(order_products.aisle_id)),STD(orders.NUM_UNIQUE(order_products.department)),STD(orders.NUM_UNIQUE(order_products.product_name)),STD(orders.NUM_UNIQUE(order_products.reordered)),STD(orders.SKEW(order_products.aisle_id)),STD(orders.SUM(order_products.aisle_id)),SUM(orders.MAX(order_products.aisle_id)),SUM(orders.MEAN(order_products.aisle_id)),SUM(orders.MIN(order_products.aisle_id)),SUM(orders.NUM_UNIQUE(order_products.department)),SUM(orders.NUM_UNIQUE(order_products.product_name)),SUM(orders.NUM_UNIQUE(order_products.reordered)),SUM(orders.SKEW(order_products.aisle_id)),SUM(orders.STD(order_products.aisle_id))
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,False,4,21,121.0,60.523810,21.0,snacks,Original Beef Jerky,1,7,10,2,0.339280,38.070486,1271.0,6.0,65.200000,23.0,4.0,6.0,2.0,0.713454,47.305038,385.0,5.250000,111.500000,60.341667,22.000000,4.000000,5.250000,1.500000,0.284635,40.106604,317.750000,5.0,91.0,53.600000,4.0,5.0,1.0,-0.050477,30.899838,268.0,1,snacks,Aged White Cheddar Popcorn,0,1,3,2015,4,1,3,2,3,4,1,2.000000,-1.885094,-0.606096,0.000000,0.000000,2.000000,0.000000,-0.832693,0.838651,0.500000,13.796135,5.399203,1.154701,0.000000,0.500000,0.577350,0.333166,50.756773,446.0,2

## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

The first thing I will do is use feeaturetools inbuilt functions to remove features with no variance and those that are highly correlated.

In [120]:
import scipy.stats as stats
# --- Write your code below ---

# The first thing I will do is use feeaturetools inbuilt functions to remove 
# features with no variance and those that are highly correlated.

# Remove features without variance
new_fm, new_features = ft.selection.remove_single_value_features(feature_matrix, features=feature_defs)

# Remove features with high correlation
new_fm, new_features = ft.selection.remove_highly_correlated_features(new_fm, features=new_features, pct_corr_threshold=0.90)

/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11

Next I will try to decide whether to use spearman's or pearson's correlation

In [121]:
# Decide whether to use spearman's or pearson's correlation
gaussian = 0
non_gaussian = 0

for column in new_fm.select_dtypes(include='number').columns:
    print(f"Analyzing {column}")

    # Statistical Test: Shapiro-Wilk Test
    stat, p_value = stats.shapiro(new_fm[column])
    print(f'Shapiro-Wilk Test for {column} - Statistic: {stat:.4f}, P-Value: {p_value:.4g}')
    
    # Interpretation
    alpha = 0.05
    if p_value > alpha:
        gaussian += 1
        print('Sample looks Gaussian (fail to reject H0)\n')
    else:
        non_gaussian += 1
        print('Sample does not look Gaussian (reject H0)\n')

print(f'Gaussian: {gaussian}, Non-Gaussian: {non_gaussian}')
print(f'Proportions: {gaussian/(gaussian+non_gaussian):.2f}, {non_gaussian/(gaussian+non_gaussian):.2f}')

Analyzing COUNT(orders)
Shapiro-Wilk Test for COUNT(orders) - Statistic: 0.7567, P-Value: 4.139e-32
Sample does not look Gaussian (reject H0)

Analyzing COUNT(order_products)
Shapiro-Wilk Test for COUNT(order_products) - Statistic: 0.8123, P-Value: 6.144e-29
Sample does not look Gaussian (reject H0)

Analyzing MAX(order_products.aisle_id)
Shapiro-Wilk Test for MAX(order_products.aisle_id) - Statistic: 0.5592, P-Value: 4.844e-40
Sample does not look Gaussian (reject H0)

Analyzing MEAN(order_products.aisle_id)
Shapiro-Wilk Test for MEAN(order_products.aisle_id) - Statistic: 0.9572, P-Value: 3.746e-14
Sample does not look Gaussian (reject H0)

Analyzing MIN(order_products.aisle_id)
Shapiro-Wilk Test for MIN(order_products.aisle_id) - Statistic: 0.6748, P-Value: 7.239e-36
Sample does not look Gaussian (reject H0)

Analyzing NUM_UNIQUE(order_products.department)
Shapiro-Wilk Test for NUM_UNIQUE(order_products.department) - Statistic: 0.9839, P-Value: 1.886e-07
Sample does not look Gaussian

From the above we can see that basically all of the features are not gaussian. Some inspection shows that those that do appear gaussian have a p-value of 1, suggesting something wrong with the execution of the test itself. These results suggest that I should not use pearsons correlation, so I will instead use spearman's rank correlation.

## Convert categorial variables via one-hot-encoding ##
The first step I will take, before computing correlations, is to one-hot-encode all of the categorical variables

In [122]:
# View all non-numerical features
non_numerical_features = new_fm.select_dtypes(exclude='number').columns
non_numerical_features

# One-hot encode the non-numerical features
one_hot_fm = pd.get_dummies(new_fm, columns=non_numerical_features)

In [123]:
one_hot_fm.head()

,COUNT(orders),COUNT(order_products),MAX(order_products.aisle_id),MEAN(order_products.aisle_id),MIN(order_products.aisle_id),NUM_UNIQUE(order_products.department),NUM_UNIQUE(order_products.product_name),NUM_UNIQUE(order_products.reordered),SKEW(order_products.aisle_id),STD(order_products.aisle_id),MAX(orders.COUNT(order_products)),MAX(orders.MEAN(order_products.aisle_id)),MAX(orders.MIN(order_products.aisle_id)),MAX(orders.NUM_UNIQUE(order_products.department)),MAX(orders.NUM_UNIQUE(order_products.reordered)),MAX(orders.SKEW(order_products.aisle_id)),MAX(orders.STD(order_products.aisle_id)),MEAN(orders.COUNT(order_products)),MEAN(orders.MAX(order_products.aisle_id)),MEAN(orders.MIN(order_products.aisle_id)),MEAN(orders.NUM_UNIQUE(order_products.reordered)),MEAN(orders.SKEW(order_products.aisle_id)),MEAN(orders.STD(order_products.aisle_id)),MIN(orders.COUNT(order_products)),MIN(orders.MAX(order_products.aisle_id)),MIN(orders.MEAN(order_products.aisle_id)),MIN(orders.NUM_UNIQUE(order_products.department)),MIN(orders.SKEW(order_products.aisle_id)),MIN(orders.STD(order_products.aisle_id)),NUM_UNIQUE(orders.MODE(order_products.department)),NUM_UNIQUE(orders.MODE(order_products.product_name)),NUM_UNIQUE(orders.MODE(order_products.reordered)),NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.WEEKDAY(order_time)),SKEW(orders.COUNT(order_products)),SKEW(orders.MAX(order_products.aisle_id)),SKEW(orders.MEAN(order_products.aisle_id)),SKEW(orders.MIN(order_products.aisle_id)),SKEW(orders.NUM_UNIQUE(order_products.department)),SKEW(orders.NUM_UNIQUE(order_products.reordered)),SKEW(orders.STD(order_products.aisle_id)),SKEW(orders.SUM(order_products.aisle_id)),STD(orders.COUNT(order_products)),STD(orders.MAX(order_products.aisle_id)),STD(orders.MEAN(order_products.aisle_id)),STD(orders.MIN(order_products.aisle_id)),STD(orders.NUM_UNIQUE(order_products.department)),STD(orders.NUM_UNIQUE(order_products.reordered)),STD(orders.SKEW(order_products.aisle_id)),SUM(orders.MIN(order_products.aisle_id)),SUM(orders.SKEW(order_products.aisle_id)),label_False,label_True,MODE(order_products.department)_alcohol,MODE(order_products.department)_babies,MODE(order_products.department)_bakery,MODE(order_products.department)_beverages,MODE(order_products.department)_breakfast,MODE(order_products.department)_canned goods,MODE(order_products.department)_dairy eggs,MODE(order_products.department)_deli,MODE(order_products.department)_frozen,MODE(order_products.department)_household,MODE(order_products.department)_international,MODE(order_products.department)_meat seafood,MODE(order_products.department)_pantry,MODE(order_products.department)_personal care,MODE(order_products.department)_pets,MODE(order_products.department)_produce,MODE(order_products.department)_snacks,MODE(order_products.product_name)_0% Fat Free Organic Milk,MODE(order_products.product_name)_0% Greek Strained Yogurt,MODE(order_products.product_name)_1 Ply Paper Towels,MODE(order_products.product_name)_1% Low Fat Milk,MODE(order_products.product_name)_1% Lowfat Milk,MODE(order_products.product_name)_1-Step 1-Minute Paneer Makhani,MODE(order_products.product_name)_100% Apple Juice,MODE(order_products.product_name)_100% Juice,"MODE(order_products.product_name)_100% Juice, Variety Pack",MODE(order_products.product_name)_100% Lactose Free Reduced Fat Milk,MODE(order_products.product_name)_100% Oatnut Bread,MODE(order_products.product_name)_100% Pineapple Juice,MODE(order_products.product_name)_100% Raw Coconut Water,MODE(order_products.product_name)_100% Recycled Bathroom Tissue,MODE(order_products.product_name)_100% Whole Wheat Bagels,MODE(order_products.product_name)_100% Whole Wheat Bread,MODE(order_products.product_name)_100% Whole Wheat English Muffin 6 Ct,MODE(order_products.product_name)_1500 Pale Ale,MODE(order_products.product_name)_2% Reduced Fat DHA Omega-3 Reduced Fat Milk,MODE(order_products.product_name)_2% Reduced Fat Milk,MODE(order_products.product_name)_2% Reduced Fat Organic Milk

Now we can compute correlations using Spearman's Rank

In [124]:
# Compute the correlations between all features and the target
corrs = one_hot_fm.corrwith(feature_matrix['label'], axis=0, method='spearman')

# Drop NaN from corrs
corrs = corrs.dropna()

# Drop the label_True and label_False columns
corrs = corrs.drop(['label_True', 'label_False'])

# Sort the correlations by magnitude
corrs = corrs.abs().sort_values(ascending=False)

/Users/MichaelKuby/miniconda3/envs/DataScienceEnv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Here we have found quite a few different features with high magnitude of correlation. Lets choose the top 10 and see how they perform on the model.

In [125]:
# Select top-10 features and return X, y (X.shape = (767, 10)
X = one_hot_fm[corrs.index[:10]]
y = feature_matrix['label']
X.shape

(767, 10)

### 2.2 Get accuracy and list features

In [126]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.78


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
1. Please list three advantages and disadvantages of featuretools.
    - Advantages of featuretools.
        1. With very little domain knowledge a data scientist is able to compute a huge number of new features from raw data, saving time and minimizing the amount of manual effort required for feature engineering.
        2. Automatically combines transformation and aggregation operations, which may lead to the discovery of highly predictive features that even a data scientist with strong domain knowledge could miss.
        3. Appears to be a scalable solution that can handle both large volumes of data and schemas that are complex.

    - Disadvantages of featuretools.
        1. Could require a huge amount of computation and memory since the process of generating all of these features has the potential to be very resource intensive.
        2. Has the potential to produce more work for the data scientist, since you are left having to decide what features to actually use. Given that the final number of features after Deep Feature Synthesis could be an order of magnitude higher than the original feature set, a data scientist may be left with a huge amount of work to determine what features are best.
        3. The creation of meaningless features. While some features created will be very meaningful, the process itself is bound to produce a large number of features that are irrelevant.
        4. (I know we were only asked for 3, but I just thought of this one...) The creation of features that are somewhat hard to interpret. Because we are combining transformations with joins on a large number of features, we may end up with something that is highly predictive but hard to understand.

2. For those disadvantages you listed above, do you have any idea to improve it?
    - One thing they have already done that is quite useful is the implementation of remove_single_value_features, remove_highly_correlated_features, remove_low_information_features, and remove_highly_null_features. This has the effect of automating away some of the task of deciding what not to use.
    - To increase the interpretability of features that are created we could use techniques like SHAP or LIME to help interpret the impact of complex features.
    - To avoid creating meaning features we could define custom primitives (transformations and aggregations) that are tailored to the specific domain.
    - After feature generation, a data scientist could use statistical tests or do EDA to filter out features with low variance or low relevance to the target.
    - To deal with the potential for high computation and memory requirements a data scientist could opt to use cloud computing resources with distributed frameworks to scale the feature generation process across multiple nodes.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 8.